# Funzioni d'utilità

In [ ]:
import os
import shutil
import json
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def remove_and_make_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        os.mkdir(path)
    else:
        os.mkdir(path)

In [ ]:
def reformat_json(path):
    with open(path, "wr") as f:
        tmp = json.loads(f.read())
        json_obj = json.dumps(tmp, indent=4)
        f.write(json_obj)
        f.close()

# Schemi mediati cluster

In [ ]:
from task_1 import make_cluster_columns_cluster
from task_2 import schema_cluster
from task_4 import make_schema_cluster

In [ ]:
clusters_datasets_base_path = "..\\..\\Dataset\\ClusterParsed\\"
clusters_info_path = "..\\DatasetSchemaMatch\\"
clusters_filename_synonym = "dic_pre_val.txt"

clusters = ".\\clusters\\"
clusters_columns = ".\\clusters\\columns\\"
clusters_dictionary = ".\\clusters\\dictionary\\"
clusters_final_dictionary = ".\\clusters\\final_dictionary\\"
clusters_schema = ".\\clusters\\schema\\"

In [ ]:
#remove_and_make_dir(clusters)

### TASK 1

In [ ]:
print("\nMake cluster columns\n")
remove_and_make_dir(clusters_columns)
make_cluster_columns_cluster(clusters_datasets_base_path, clusters_info_path, clusters_filename_synonym, clusters_columns)

### TASK 2

In [ ]:
print("\nExtract schema\n")
remove_and_make_dir(clusters_dictionary)
start = time.time()
schema_cluster(clusters_columns, clusters_dictionary, 0.1, ".txt")
end = time.time()
print("\nTempo di esecuzione: "+str(end-start)+" (s)"+"\n")

### TASK 4

In [ ]:
print("\nMake schemas\n")
remove_and_make_dir(clusters_schema)
make_schema_cluster(clusters_datasets_base_path, clusters_final_dictionary, clusters_schema, ".txt")

# Schema mediato finale

In [ ]:
from task_1 import make_cluster_columns_final
from task_2 import schema_final
from task_4 import make_schema_final

In [ ]:
final_datasets_base_path = ".\\clusters\\schema\\"
final_info_path = ".\\files_matching\\files_vari"
final_filename_synonym = "dic_pre_val.txt"

final_schema = ".\\final_schema\\"
final_schema_columns = ".\\final_schema\\columns\\"
final_schema_dictionary = ".\\final_schema\\dictionary\\"
final_schema_final_dictionary = ".\\final_schema\\final_dictionary\\"
final_schema_schema = ".\\final_schema\\schema\\"

In [ ]:
remove_and_make_dir(final_schema)

### TASK 1

In [ ]:
print("\nMake cluster columns\n")
remove_and_make_dir(final_schema_columns)
make_cluster_columns_final(final_datasets_base_path, final_info_path, final_filename_synonym, final_schema_columns)

### TASK 2

In [ ]:
print("\nExtract schema\n")SS
remove_and_make_dir(final_schema_dictionary)
schema_final(final_schema_columns, final_schema_dictionary, 0.1, ".txt")

### TASK 4

In [ ]:
print("\nMake schema\n")
remove_and_make_dir(final_schema_schema)
make_schema_final(final_datasets_base_path, final_schema_final_dictionary, final_schema_schema, ".json")